In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('../data/Train_dataset.csv')
train.head(1)

In [ ]:
from kp_auto_ml import pre_processing as pp

In [ ]:
train.columns = train.columns.str.lower().str.replace(' ', '_')

In [ ]:
train.head(1)

In [ ]:
label_encode:list[pp.PreLabelEncoderConfig] = []
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='city',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.city.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_location',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_location.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='endorsed_by',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.endorsed_by.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_theme',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_theme.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_type',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_type.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='cuisine',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.cuisine.value_counts().keys())}))
    )

numeric_col_changer:list[pp.PreNumericColDataChangeConfig] = []
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='facebook_popularity_quotient',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='instagram_popularity_quotient',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='overall_restaurant_rating',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='live_music_rating',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='ambience',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='resturant_tier',data_type=int))

model_config = pp.PreProcessingConfig(
    encoding_dummies=[],
    exclude_columns=['opening_day_of_restaurant','registration_number'],
    label_encode=label_encode,
    numeric_cols_data_changer=numeric_col_changer,
    target_column='annual_turnover'
)

In [ ]:
pre_processed_df = pp.process(train,model_config)

In [ ]:
from kp_auto_ml import model_training_data_prep as dp
from kp_auto_ml import model_training_helper as mth
from kp_auto_ml import model_list_helper as mlh

In [ ]:
data = dp.ModelTrainingData(pre_processed_df, dp.ScalerType.QUANTILE_TRANSFORMER, .92,use_pca=True,use_polynomials=True,use_feature_selection=True)


trainer = mth.ModelTrainer(data=data)

In [ ]:
trainer.perform_operation_regression(exclude_models=[
    mlh.ModelAndParam.SVR_Regression
    ,mlh.ModelAndParam.GradientBoosting_Regressor
    ,mlh.ModelAndParam.DecisionTree_Regressor
    ,mlh.ModelAndParam.RandomForest_Regressor
    ],permutate_n_less_column=0)

In [ ]:
trainer.performance_df.loc[trainer.performance_df['score'].idxmax()]
# trainer.performance_df

In [ ]:
test = pd.read_csv('../data/Test_dataset.csv')

In [ ]:
test.columns = test.columns.str.lower().str.replace(' ', '_')

In [ ]:
test.columns[6]

In [ ]:
new_column_name = label_encode[2].column_name
test = test.rename(columns={'endoresed_by': new_column_name})

In [ ]:
label_encode[0] = pp.PreLabelEncoderConfig(
    column_name='city',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(test.city.value_counts().keys())}))

In [ ]:
model_config_test = pp.PreProcessingConfig(
    encoding_dummies=[],
    exclude_columns=['opening_day_of_restaurant','registration_number'],
    label_encode=label_encode,
    numeric_cols_data_changer=numeric_col_changer,
    target_column=''
)

In [ ]:
test = pp.process_test(test,model_config_test)

In [ ]:
test.city.value_counts()

In [ ]:
data.transform_test_data(test)

In [ ]:
from kp_auto_ml import model_feature_selector as mfs

In [ ]:
mfs.Run_Features_Selection(data.X_original,data.Y)

In [ ]:
mfs.DecisionTreeClassifierFeaturesSelection(data.X_original,data.Y)

In [ ]:
mfs.LassoFeaturesSelection(data.X_original,data.Y)